In [ ]:
#| default_exp retrieve_last_01

In [ ]:
#| export
import pandas as pd
import requests
import boto3
import json

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
def last_cred():

    secret_name = "last_keys"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    last_key = json.loads(get_secret_value_response['SecretString'])['last_apiKEY']
    last_agent = json.loads(get_secret_value_response['SecretString'])['last_userAGENT']
    last_user = json.loads(get_secret_value_response['SecretString'])['last_username']

    return last_key, last_agent, last_user

In [ ]:
#| export
def last_get(method, agent, user, key, period='1month'):

    headers = {
    'user-agent': agent
    }

    payload = {'method': method, 
                'period': period,
            'user': user,
            'api_key': key,
            'format': 'json'
                }


    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    # print(r.status_code)
    return r.json()

In [ ]:
#| export
def last_format(f_tracks, cutoff):
    f_tracks = pd.DataFrame(f_tracks['toptracks']['track'])
    f_tracks = f_tracks[['name', 'artist', 'playcount']]
    f_tracks['artist'] = f_tracks['artist'].apply(lambda x: x['name'])
    f_tracks['playcount'] = f_tracks['playcount'].astype(int)
    f_tracks = f_tracks.sort_values('playcount', ascending=False)
    f_tracks = f_tracks[f_tracks['playcount'] >= cutoff]

    return f_tracks

In [ ]:
#| export
if __name__ == '__main__':
    API_KEY, USER_AGENT, USERNAME = last_cred()
    tracks = last_get('user.gettoptracks', USER_AGENT, USERNAME, API_KEY)
    formatted = last_format(tracks, 7)
    formatted.to_csv('s3://spotify-net/df_tracks.csv')
    print('Retrieved')

Retrieved
